In [1]:
from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

In [5]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/experiments")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [13]:
# Deleting all cell data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Calculation
                OR n:StereologyDetail
                OR n:Distribution
                OR n:CellMorphology 
                OR n:Quantitation
                OR n:Analysis
                OR n:Experiment
                OR n:SectioningDetail
                OR n:SectioningInstrument
                OR n:ReporterIncubation
                OR n:Reporter
                OR n:ReporterLabel
                OR n:ReporterTarget
                OR n:ReporterType
                OR n:VisualizationProtocol
                OR n:Solution
                OR n:Software
                OR n:Microscope
                OR n:ElectronMicroscopeDetail
                OR n:LightFluorescenceMicroscopeDetail
            DETACH DELETE n
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:Distribution) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:CellMorphology) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Quantitation) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Analysis) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Experiment) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Cell(Distribution)
            DROP INDEX ON :CellType(CellMorphology)
            DROP INDEX ON :CellPhenotype(Quantitation)
            DROP INDEX ON :CellPhenotypeCategory(Analysis)
            DROP INDEX ON :NeuralStructure(Experiment)
        ''')
        print("wipeddatabase")

wipeddatabase


In [14]:
# Adding Calculation to graph from calculations.csv
csv_file_path = "file:///%s" % get_csv_path("calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Calculation { id: row.ID, description: row.Calculation_performed})
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_type("Calculation", session)

Added 958 nodes of type Calculation


In [15]:
# Adding StereologyDetail to graph from stereology_details.csv
csv_file_path = "file:///%s" % get_csv_path("stereology_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:StereologyDetail { id: row.ID, name: row.Stereology_details_name, probe: row.Probe, identificationFeature: row.Identification_feature, disectorHeight: toInt(row.Disector_height), areaSubfraction: toFloat(row.Area_subfraction), heightSubfraction: toFloat(row.Height_subfraction), investigatedSections: toFloat(row.Investigated_sections), investigatedFields: toInt(row.Investigated_fields), countedObjects: toInt(row.Counted_objects), coefficientOfError: toFloat(row.Coefficient_of_error), estimatedVolume: toFloat(row.Estimated_volume), volumeUnit: row.Volume_unit, anyExceptProbe: toInt(row.any_except_probe)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("StereologyDetail", session)

Added 212 nodes of type StereologyDetail


In [16]:
# Adding Distribution to graph from distributions.csv
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Distribution { id: row.ID, name: row.Distribution_name, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, distribution: row.Distribution, distributionDimensions: row.Distribution_dimensions, analysisTypePrimary: row.Analysis_type_primary, analysisTypeSecondary: row.Analysis_type_secondary})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Distribution(name)")
    session.run("CREATE CONSTRAINT ON (n:Distribution) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Distribution", session)

Added 18 nodes of type Distribution


In [17]:
# Adding CellMorphology to graph from cell_morphologies.csv
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellMorphology { id: row.ID, name: row.Morphology_name, neuromorphoId: row.Neuromorpho_ID, somaSurface: toFloat(row.Soma_surface), numberOfStems: toInt(row.Number_stems), numberOfBifurcations: toInt(row.Number_bifurcations), numberOfBranches: toInt(row.Number_branches), overallWidth: toFloat(row.Overall_width), overallHeight: toFloat(row.Overall_height), overallDepth: toFloat(row.Overall_depth), averageBranchDiameter: toFloat(row.Average_branch_diameter), averageContraction: toFloat(row.Average_contraction), totalArborLength: toFloat(row.Total_arbor_length), totalArborSurface: toFloat(row.Total_arbor_surface), totalArborVolume: toFloat(row.Total_arbor_volume), maxEuclideanDistance: toFloat(row.Max_Euclidean_distance), maxPathDistance: toFloat(row.Max_path_distance), maxBranchOrder: toInt(row.Max_branch_order), totalFragmentation: toInt(row.Total_fragmentation), partitionAsymmetry: toFloat(row.Partition_asymmetry), averageRalls: toInt(row.Average_Ralls), averageBifurcationAngleLocal: toFloat(row.Average_bifurcation_angle_local), averageBifurcationAngleRemote: toFloat(row.Average_bifurcation_angle_remote), fractalDimension: toFloat(row.Fractal_dimension), physicalIntegrity: row.Physical_integrity, structuralDomains: row.Structural_domains, morphologicalAttributes: row.Morphological_attributes, originalFormat: row.Original_format})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :CellMorphology(name)")
    session.run("CREATE CONSTRAINT ON (n:CellMorphology) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("CellMorphology", session)

Added 50 nodes of type CellMorphology


In [18]:
# Adding Quantitation to graph from cell_morphologies.csv
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Quantitation { id: row.ID, name: row.Quantitation_name, estimateRelevance: row.Estimate_relevance, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, originalExtent: row.Original_extent, number: toInt(row.Number), numberSD: toInt(row.Number_SD), density: toFloat(row.Density), densityUnit: row.Density_unit, densitySD: toInt(row.Density_SD), volumetricDensity: toInt(row.Volumetric_density), estimateExtraction: row.Estimate_extraction})
    """ % csv_file_path

with driver.session() as session: 
    session.run("CREATE INDEX ON :Quantitation(name)")
    session.run("CREATE CONSTRAINT ON (n:Quantitation) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Quantitation", session)

Added 1074 nodes of type Quantitation


In [19]:
# Adding Analysis to graph from derived_data_records.csv
csv_file_path = "file:///%s" % get_csv_path("derived_data_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Analysis { id: row.ID, name: row.Derived_data_record_name, numberOfAnimals: row.Number_of_animals, shrinkageCorrection: row.Shrinkage_correction })
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE INDEX ON :Analysis(name)")
    session.run("CREATE CONSTRAINT ON (n:Analysis) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Analysis", session)

Added 668 nodes of type Analysis


In [20]:
# Adding Experiment to graph from experiments.csv
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Experiment { id: row.ID, name: row.Experiment_name, animalStatus: row.Animal_status, ageLowerLimit: toInt(row.Age_lower_limit), ageUpperLimit: toInt(row.Age_upper_limit), weightLowerLimit: toInt(row.Weight_lower_limit), weightUpperLimit: toInt(row.Weight_upper_limit) })
    """ % csv_file_path

with driver.session() as session: 
    session.run("CREATE INDEX ON :Experiment(name)")
    session.run("CREATE CONSTRAINT ON (n:Experiment) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Experiment", session)

Added 375 nodes of type Experiment


In [21]:
# Adding SectioningDetail to graph from sectioning_details.csv
csv_file_path = "file:///%s" % get_csv_path("sectioning_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningDetail { id: row.ID, sectionThickness: toFloat(row.Section_thickness), sectionOrientation: row.Section_orientation })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningDetail", session)

Added 510 nodes of type SectioningDetail


In [22]:
# Adding SectioningInstrument to graph from sectioning_instruments.csv
csv_file_path = "file:///%s" % get_csv_path("sectioning_instruments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningInstrument { id: row.ID, name: row.Sectioning_instrument_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningInstrument", session)

Added 6 nodes of type SectioningInstrument


In [23]:
# Adding ReporterIncubation to graph from reporter_incumations.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_incubations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterIncubation { id: row.ID, order: row.Order, concentration: row.Concentration, time: toFloat(row.Time), temperature: toInt(row.temperature) })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterIncubation", session)

Added 713 nodes of type ReporterIncubation


In [24]:
# Adding Reporter to graph from reporters.csv
csv_file_path = "file:///%s" % get_csv_path("reporters.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Reporter { id: row.ID, name: row.Name, type: row.Reporter_type, uniqueId: row.Reporter_unique_ID, comment: row.Comment, originSpecie: row.Origin_species  })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Reporter", session)

Added 249 nodes of type Reporter


In [25]:
# Adding ReporterLabel to graph from reporter_labels.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_labels.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterLabel { id: row.ID, name: row.Label})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterLabel", session)

Added 23 nodes of type ReporterLabel


In [26]:
# Adding ReporterTarget to graph from reporter_targets.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_targets.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterTarget { id: row.ID, phenotype: row.Phenotype})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterTarget", session)

Added 100 nodes of type ReporterTarget


In [27]:
# Adding VisualizationProtocol to graph from visualization_protocols.csv
csv_file_path = "file:///%s" % get_csv_path("visualization_protocols.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:VisualizationProtocol { id: row.ID, name: row.Protocol_name, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("VisualizationProtocol", session)

Added 12 nodes of type VisualizationProtocol


In [28]:
# Adding Solution to graph from solutions.csv
csv_file_path = "file:///%s" % get_csv_path("solutions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Solution { id: row.ID, name: row.Solution_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Solution", session)

Added 266 nodes of type Solution


In [29]:
# Adding Microscope to graph from microscopes.csv
csv_file_path = "file:///%s" % get_csv_path("microscopes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Microscope { id: row.ID, type: row.Microscope_type, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Microscope", session)

Added 9 nodes of type Microscope


In [30]:
# Adding ElectronMicroscopeDetail to graph from electron_microscopy_details.csv
csv_file_path = "file:///%s" % get_csv_path("electron_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ElectronMicroscopeDetail { id: row.ID, name: row.EM_details_name, gridType: row.Grid_type, magnification: row.Magnification })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ElectronMicroscopeDetail", session)

Added 27 nodes of type ElectronMicroscopeDetail


In [31]:
# Adding LightFluorescenceMicroscopeDetail to graph from light_fluorescence_microscopy_details.csv
csv_file_path = "file:///%s" % get_csv_path("light_fluorescence_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:LightFluorescenceMicroscopeDetail { id: row.ID, name: row.Microscopy_details_name, mountingMedium: toInt(row.Mounting_medium), refractionMedium: row.Refraction_medium, numericalAperature: toFloat(row.Numerical_aperture), objectiveLens: row.Objective_lens, totalMagnification: row.Total_magnification, pixelSize: toFloat(row.Pixel_size), z_stack: toBoolean(row.Z_stack), opticalSliceSize: row.Optical_slice_size })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("LightFluorescenceMicroscopeDetail", session)

Added 238 nodes of type LightFluorescenceMicroscopeDetail


In [32]:
# Adding Software to graph from softwares.csv
csv_file_path = "file:///%s" % get_csv_path("softwares.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Software { id: row.ID, name: row.Software_name, rrid: row.Software_RRID })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Software", session)

Added 50 nodes of type Software


In [33]:
## Microscope RELATIONS

# :ElectronMicroscopeDetail - :MICROSCOPE_TYPE - :Microscope
csv_file_path = "file:///%s" % get_csv_path("electron_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ElectronMicroscopeDetail { id: row.ID})
        MATCH (c:Microscope { id: row.Microscope })
        MERGE (a)-[:MICROSCOPE_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE_TYPE", "ElectronMicroscopeDetail", "Microscope", session)
    
# :LightFluorescenceMicroscopeDetail - :MICROSCOPE_TYPE - :Microscope
csv_file_path = "file:///%s" % get_csv_path("light_fluorescence_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:LightFluorescenceMicroscopeDetail { id: row.ID})
        MATCH (c:Microscope { id: row.Microscope })
        MERGE (a)-[:MICROSCOPE_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE_TYPE", "LightFluorescenceMicroscopeDetail", "Microscope", session)

# :Analysis - :MICROSCOPE - :ElectronMicroscopeDetail
csv_file_path = "file:///%s" % get_csv_path("derived_data_EMdetails.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:ElectronMicroscopeDetail { id: row.EMdetails })
        MERGE (a)-[:MICROSCOPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE", "Analysis", "ElectronMicroscopeDetail", session)

# :Analysis - :MICROSCOPE - :LightFluorescenceMicroscopeDetail
csv_file_path = "file:///%s" % get_csv_path("derived_data_LFMdetails.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record_ID})
        MATCH (c:LightFluorescenceMicroscopeDetail { id: row.LFMdetails_ID })
        MERGE (a)-[:MICROSCOPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE", "Analysis", "LightFluorescenceMicroscopeDetail", session)


Added 27 relationships of type ElectronMicroscopeDetail MICROSCOPE_TYPE Microscope
Added 238 relationships of type LightFluorescenceMicroscopeDetail MICROSCOPE_TYPE Microscope
Added 59 relationships of type Analysis MICROSCOPE ElectronMicroscopeDetail
Added 538 relationships of type Analysis MICROSCOPE LightFluorescenceMicroscopeDetail


In [34]:
## Reporter RELATIONS
csv_file_path = "file:///%s" % get_csv_path("reporters.csv")

## :Reporter - :LABEL - :ReporterLabel
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterLabel { id: row.Label })
        MERGE (a)-[:LABEL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("LABEL", "Reporter", "ReporterLabel", session)
    
## :Reporter - :TARGET - :ReporterTarget
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterTarget { id: row.Target })
        MERGE (a)-[:TARGET]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET", "Reporter", "ReporterTarget", session)

Added 246 relationships of type Reporter LABEL ReporterLabel
Added 241 relationships of type Reporter TARGET ReporterTarget


In [35]:
## SectioningDetail RELATIONS
csv_file_path = "file:///%s" % get_csv_path("sectioning_details.csv")

## :SectioningDetail - :SECTIONED_BY - :SectioningInstrument
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:SectioningDetail { id: row.ID})
        MATCH (c:SectioningInstrument { id: row.Sectioning_instrument })
        MERGE (a)-[:SECTIONED_BY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECTIONED_BY", "SectioningDetail", "SectioningInstrument", session)


Added 382 relationships of type SectioningDetail SECTIONED_BY SectioningInstrument


In [36]:
## ReporterIncubation RELATIONS
csv_file_path = "file:///%s" % get_csv_path("reporter_incubations.csv")

## :ReporterIncubation - :REPORTER - :Reporter
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ReporterIncubation { id: row.ID})
        MATCH (c:Reporter { id: row.Reporter })
        MERGE (a)-[:REPORTER]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REPORTER", "ReporterIncubation", "Reporter", session)

Added 713 relationships of type ReporterIncubation REPORTER Reporter


In [38]:
### Experiment RELATIONS
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")

## :Experiment - :ANAESTHETIC - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Anaesthetic })
        MERGE (a)-[:ANAESTHETIC]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ANAESTHETIC", "Experiment", "Solution", session)
    
## :Experiment - :PERFUSION_FIX_MEDIUM - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Perfusion_fix_medium })
        MERGE (a)-[:PERFUSION_FIX_MEDIUM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFUSION_FIX_MEDIUM", "Experiment", "Solution", session)

## :Experiment - :DATA_ORIGIN - :Source
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Source { id: row.Source })
        MERGE (a)-[:DATA_ORIGIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_ORIGIN", "Experiment", "Source", session)


Added 313 relationships of type Experiment ANAESTHETIC Solution
Added 312 relationships of type Experiment PERFUSION_FIX_MEDIUM Solution
Added 375 relationships of type Experiment DATA_ORIGIN Source


In [39]:
### Analysis RELATIONS 
csv_file_path = "file:///%s" % get_csv_path("derived_data_records.csv")

## :Analysis - :SPECIMEN - :Specimen 
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:Specimen { id: row.Specimen })
        MERGE (a)-[:SPECIMEN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIMEN", "Analysis", "Specimen", session)
    
## :Analysis - :OBJECT_OF_INTEREST - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:NeuralStructure { id: row.Object_of_interest })
        MERGE (a)-[:OBJECT_OF_INTEREST]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("OBJECT_OF_INTEREST", "Analysis", "NeuralStructure", session)
    
## Add property to :Analysis - :OBJECT_OF_INTEREST - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis)-[r:OBJECT_OF_INTEREST]-(b:NeuralStructure)
        WHERE row.OOI_recognition_criteria IS NOT NULL
        SET r.recognitionCriteria = row.OOI_recognition_criteria
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
## :Analysis - :CELL_TYPE_PUTATIVE - :Cell
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:Cell { id: row.Cell_type_putative })
        MERGE (a)-[:CELL_TYPE_PUTATIVE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_TYPE_PUTATIVE", "Analysis", "Cell", session)
    
## :Analysis - :VISUALIZATION_METHOD - :VisualizationProtocol
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:VisualizationProtocol { id: row.Visualization_method })
        MERGE (a)-[:VISUALIZATION_METHOD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("VISUALIZATION_METHOD", "Analysis", "VisualizationProtocol", session)


Added 668 relationships of type Analysis SPECIMEN Specimen
Added 668 relationships of type Analysis OBJECT_OF_INTEREST NeuralStructure
Added 668 relationships of type Analysis CELL_TYPE_PUTATIVE Cell
Added 666 relationships of type Analysis VISUALIZATION_METHOD VisualizationProtocol


In [40]:
### Quantitation RELATIONS

## :Analysis - :DATA_TYPE - :Quantitation
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Quantitation { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Quantitation", session)
    
## :Quantitation - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Quantitation", "RegionRecord", session)
    
## :Quantitation - :REGION_ZONE - :RegionZone
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "Quantitation", "RegionZone", session)
    
## :Quantitation - :CELLULAR_TARGET_REGION - :CellularRegion
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_target_region })
        MERGE (a)-[:CELLULAR_TARGET_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_TARGET_REGION", "Quantitation", "CellularRegion", session)
    
## :Quantitation - :TARGET_CELL - :Cell
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Cell { id: row.Cellular_target_ID })
        MERGE (a)-[:TARGET_CELL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET_CELL", "Quantitation", "Cell", session)

## :Quantitation - :STEREOLOGY - :StereologyDetail
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Quantitation", "StereologyDetail", session)
    
## :Quantitation - :SOFTWARE - :Software
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:SOFTWARE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SOFTWARE", "Quantitation", "Software", session)
    
## :Quantitation - :CALCULATION - :Calculation
csv_file_path = "file:///%s" % get_csv_path("calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.Quantitation_ID})
        MATCH (c:Calculation { id: row.ID })
        MERGE (a)-[:CALCULATION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CALCULATION", "Quantitation", "Calculation", session)

Added 1074 relationships of type Analysis DATA_TYPE Quantitation
Added 1074 relationships of type Quantitation REGION_RECORD RegionRecord
Added 453 relationships of type Quantitation REGION_ZONE RegionZone
Added 102 relationships of type Quantitation CELLULAR_TARGET_REGION CellularRegion
Added 15 relationships of type Quantitation TARGET_CELL Cell
Added 418 relationships of type Quantitation STEREOLOGY StereologyDetail
Added 658 relationships of type Quantitation SOFTWARE Software
Added 958 relationships of type Quantitation CALCULATION Calculation


In [41]:
### CellMorphology RELATIONS

## :Analysis - :DATA_TYPE - :CellMorphology
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:CellMorphology { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "CellMorphology", session)

## :CellMorphology - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "CellMorphology", "RegionRecord", session)
    
## :CellMorphology - :REGION_ZONE - :RegionZone
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "CellMorphology", "RegionZone", session)
    
## :CellMorphology - :RECONSTRUCTED_WITH - :Software
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:Software { id: row.Reconstruction_method })
        MERGE (a)-[:RECONSTRUCTED_WITH]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RECONSTRUCTED_WITH", "CellMorphology", "Software", session)

Added 50 relationships of type Analysis DATA_TYPE CellMorphology
Added 50 relationships of type CellMorphology REGION_RECORD RegionRecord
Added 37 relationships of type CellMorphology REGION_ZONE RegionZone
Added 50 relationships of type CellMorphology RECONSTRUCTED_WITH Software


In [42]:
### DISTRIBUTION RELATIONS

## :Analysis - :DATA_TYPE - :Distribution
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Distribution { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Distribution", session)

## :Distribution - :PERFORMED_ON - :Software    <- TODO FIGURE OUT WHY THESE ARE NOT ADDEDD
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:PERFORMED_ON]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFORMED_ON", "Distribution", "Software", session)
    
## :Distribution - :CELLULAR_REGION - :CellularRegion
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_region })
        MERGE (a)-[:CELLULAR_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
# Add property to :relation
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Cellular_region_cytochemical_ID is not null
        MATCH (a:Distribution {id: row.ID})-[r:CELLULAR_REGION]-(b:CellularRegion)
        SET r.cytochemicalId = row.Cellular_region_cytochemical_ID
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_REGION", "Distribution", "CellularRegion", session)

## :Distribution - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Distribution", "RegionRecord", session)

## :Distribution - :STEREOLOGY - :StereologyDetail
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Distribution", "StereologyDetail", session)
    
## :Distribution - :RELATED_TO - :Quantitation
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Quantitation { id: row.Related_quantitation })
        MERGE (a)-[:RELATED_TO]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RELATED_TO", "Distribution", "Quantitation", session)



Added 18 relationships of type Analysis DATA_TYPE Distribution
Added 10 relationships of type Distribution PERFORMED_ON Software
Added 4 relationships of type Distribution CELLULAR_REGION CellularRegion
Added 18 relationships of type Distribution REGION_RECORD RegionRecord
Added 4 relationships of type Distribution STEREOLOGY StereologyDetail
Added 16 relationships of type Distribution RELATED_TO Quantitation


In [43]:
### Derived relations from Experiments and Analysis (DDR) and Specimen

## :Analysis - :SECTIONING_DETAIL - :SectioningDetail
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_sectioning_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:SectioningDetail { id: row.sectioningDetailsId })
        MERGE (a)-[:SECTIONING_DETAIL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECTIONING_DETAIL", "Analysis", "SectioningDetail", session)

    
## :Analysis - :REPORTER_INCUBATION - :ReporterIncubation
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_reporters.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:ReporterIncubation { id: row.reporterId })
        MERGE (a)-[:REPORTER_INCUBATION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REPORTER_INCUBATION", "Analysis", "ReporterIncubation", session)
    
## :Analysis - :PART_OF - :Experiment
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_experiments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:Experiment { id: row.experimentId })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", "Analysis", "Experiment", session)



Added 610 relationships of type Analysis SECTIONING_DETAIL SectioningDetail
Added 906 relationships of type Analysis REPORTER_INCUBATION ReporterIncubation
Added 668 relationships of type Analysis PART_OF Experiment
